In [10]:
import altair as alt
from vega_datasets import data
from IPython.display import display

# Load data
iris = data.iris()

# Create chart
chart = alt.Chart(iris).mark_circle().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
)

# Display chart in HTML
chart.save('chart.json')
display(chart)

alt.Chart(...)

In [9]:
import pandas as pd

source = pd.DataFrame(
    [
        {"a": "a1", "b": "b1", "c": "x", "p": "0.14"},
        {"a": "a1", "b": "b1", "c": "y", "p": "0.60"},
        {"a": "a1", "b": "b1", "c": "z", "p": "0.03"},
        {"a": "a1", "b": "b2", "c": "x", "p": "0.80"},
        {"a": "a1", "b": "b2", "c": "y", "p": "0.38"},
        {"a": "a1", "b": "b2", "c": "z", "p": "0.55"},
        {"a": "a1", "b": "b3", "c": "x", "p": "0.11"},
        {"a": "a1", "b": "b3", "c": "y", "p": "0.58"},
        {"a": "a1", "b": "b3", "c": "z", "p": "0.79"},
        {"a": "a2", "b": "b1", "c": "x", "p": "0.83"},
        {"a": "a2", "b": "b1", "c": "y", "p": "0.87"},
        {"a": "a2", "b": "b1", "c": "z", "p": "0.67"},
        {"a": "a2", "b": "b2", "c": "x", "p": "0.97"},
        {"a": "a2", "b": "b2", "c": "y", "p": "0.84"},
        {"a": "a2", "b": "b2", "c": "z", "p": "0.90"},
        {"a": "a2", "b": "b3", "c": "x", "p": "0.74"},
        {"a": "a2", "b": "b3", "c": "y", "p": "0.64"},
        {"a": "a2", "b": "b3", "c": "z", "p": "0.19"},
        {"a": "a3", "b": "b1", "c": "x", "p": "0.57"},
        {"a": "a3", "b": "b1", "c": "y", "p": "0.35"},
        {"a": "a3", "b": "b1", "c": "z", "p": "0.49"},
        {"a": "a3", "b": "b2", "c": "x", "p": "0.91"},
        {"a": "a3", "b": "b2", "c": "y", "p": "0.38"},
        {"a": "a3", "b": "b2", "c": "z", "p": "0.91"},
        {"a": "a3", "b": "b3", "c": "x", "p": "0.99"},
        {"a": "a3", "b": "b3", "c": "y", "p": "0.80"},
        {"a": "a3", "b": "b3", "c": "z", "p": "0.37"},
    ]
)

chart = alt.Chart(source, width=60, height=alt.Step(8)).mark_bar().encode(
    y=alt.Y("c:N", axis=None),
    x=alt.X("p:Q", title=None, axis=alt.Axis(format="%")),
    color=alt.Color(
        "c:N", title="settings", legend=alt.Legend(orient="bottom", titleOrient="left")
    ),
    row=alt.Row("a:N", title="Factor A", header=alt.Header(labelAngle=0)),
    column=alt.Column("b:N", title="Factor B"),
)
chart.save('chart.json')
display(chart)

alt.Chart(...)

In [8]:
from vega_datasets import data

# Import example data
source = data.stocks()

# Create a common chart object
# Use `transform_filter` to reduce the dataset to clarify our example. Not required.
chart = (
    alt.Chart(source)
    .transform_filter(alt.datum.symbol != "IBM")
    .encode(color=alt.Color("symbol", legend=None))
)

# Draw the line
line = chart.mark_line().encode(x="date:T", y="price:Q")

# Use the `argmax` aggregate to limit the dataset to the final value
label = chart.encode(
    x=alt.X("max(date):T"),
    y=alt.Y("price:Q", aggregate=alt.ArgmaxDef(argmax="date")),
    text="symbol",
)

# Create a text label
text = label.mark_text(align="left", dx=4)

# Create a circle annotation
circle = label.mark_circle()

# Draw the chart with all the layers combined
chart = line + circle + text
chart.save('chart.json')
display(chart)

alt.LayerChart(...)

In [13]:
from vega_datasets import data

source = data.stocks()

chart = alt.Chart(source).transform_filter(
    alt.datum.symbol != 'GOOG'
).mark_area().encode(
    x='date:T',
    y='price:Q',
    color='symbol:N',
    row=alt.Row('symbol:N', sort=['MSFT', 'AAPL', 'IBM', 'AMZN'])
).properties(height=50, width=400)
chart.save('chart.json')
display(chart)

alt.Chart(...)

In [ ]:
print('test')